# Анализ NPS телекоммуникационной компании
Определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

# Выгрузка таблиц

In [140]:
import pandas as pd
from sqlalchemy import create_engine

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 300

try:
    path_to_db = '/datasets/telecomm_csi.db' # путь к файлу
    engine = create_engine(f'sqlite:///{path_to_db}', echo = False)
except:
    path_to_db = 'telecomm_csi.db' # путь к файлу
    engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

user = '''
select * 
from user;
'''
location = '''
select * 
from location;
'''
age_segment = '''
select * 
from age_segment;
'''
traffic_segment = '''
select * 
from traffic_segment;
'''
lifetime_segment = '''
select * 
from lifetime_segment;
'''

user = pd.read_sql(user, engine)
location = pd.read_sql(location, engine)
age_segment = pd.read_sql(age_segment, engine)
traffic_segment = pd.read_sql(traffic_segment, engine)
lifetime_segment = pd.read_sql(lifetime_segment, engine)
    


# Анализ

In [141]:
user.head()
location.head()
age_segment.head()
traffic_segment.head()
lifetime_segment.head()

,user_id,lt_day,age,gender_segment,os_name,cpe_type_name,location_id,age_gr_id,tr_gr_id,lt_gr_id,nps_score
0,A001A2,2320,45.0,1.0,ANDROID,SMARTPHONE,55,5,5,8,10
1,A001WF,2344,53.0,0.0,ANDROID,SMARTPHONE,21,5,5,8,10
2,A003Q7,467,57.0,0.0,ANDROID,SMARTPHONE,28,6,9,6,10
3,A004TB,4190,44.0,1.0,IOS,SMARTPHONE,38,4,4,8,10
4,A004XT,1163,24.0,0.0,ANDROID,SMARTPHONE,39,2,6,8,10


,location_id,city,country
0,1,Архангельск,Россия
1,2,Астрахань,Россия
2,3,Балашиха,Россия
3,4,Барнаул,Россия
4,5,Белгород,Россия


,age_gr_id,bucket_min,bucket_max,title
0,1,NaN,15.0,01 до 16
1,2,16.0,24.0,02 16-24
2,3,25.0,34.0,03 25-34
3,4,35.0,44.0,04 35-44
4,5,45.0,54.0,05 45-54


,tr_gr_id,bucket_min,bucket_max,title
0,1,0.00,0.00,01 0
1,2,0.00,0.01,01 0-0.01
2,3,0.01,0.10,02 0.01-0.1
3,4,0.10,1.00,03 0.1-1
4,5,1.00,5.00,04 1-5


,lt_gr_id,bucket_min,bucket_max,title
0,1,1.0,1.0,01 1
1,2,2.0,2.0,02 2
2,3,3.0,3.0,03 3
3,4,4.0,6.0,04 4-6
4,5,7.0,12.0,05 7-12


In [142]:
query = '''
select u.user_id, u.lt_day, 
case
    when u.lt_day > 365 then 0
    else 1
end is_new, u.age, 
case
    when u.gender_segment = 1 then 'Ж'
    else 'М'
end gender_segment, u.os_name, u.cpe_type_name, l.country, l.city, substr(a.title, 4, 8) age_segment,
substr(t.title, 4, 8) traffic_segment, substr(lt.title, 4, 8) lifetime_segment, u.nps_score,
case
    when u.nps_score between 0 and 6 then 'критики'
    when u.nps_score between 7 and 8 then 'нейтралы'
    when u.nps_score between 9 and 10 then 'cторонники'
end nps_group
from user u
inner join location l on u.location_id = l.location_id
inner join age_segment a on u.age_gr_id = a.age_gr_id
inner join traffic_segment t on u.tr_gr_id = t.tr_gr_id
inner join lifetime_segment lt on u.lt_gr_id = lt.lt_gr_id
'''

df = pd.read_sql(query, engine)

In [143]:
df = df[~df['age'].isna()].reset_index(drop = True)
df.head()

# Tableau

https://public.tableau.com/views/telecom_16335125974790/Story1?:language=en-US&:display_count=n&:origin=viz_share_link

# Presentation

https://docs.google.com/presentation/d/1FsLCqNu5WBP8uAV4Q5zoys07okBqVM2qjw3YSyOsrYA/edit?usp=sharing